<a href="https://colab.research.google.com/github/leman-cap13/kaggle_datasets_/blob/main/data_analitics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!wget https://raw.githubusercontent.com/Rstam59/TaskDataRepoForStudents/refs/heads/main/heart-disease.csv

In [ ]:
df=pd.read_csv('heart-disease.csv')
df

In [ ]:
X=df.drop('target',axis=1)
y=df['target'].copy()
# X boyuk herfle yazilir cunki matrixi temsil edir
# y balaca yazilir cunki vektoru temsil edir
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=(0.2),random_state=13)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf=RandomForestClassifier(random_state=13)
forest_clf.fit(X_train,y_train)

In [ ]:
forest_clf.score(X_train,y_train)

In [ ]:
forest_clf.score(X_test,y_test)

In [ ]:
forest_clf.predict(X_test.iloc[[0]]) # array 0 saglam array 1 urek xesteliyi var

In [ ]:
X_test.iloc[[0]]

In [ ]:
y_test.iloc[[0]]

In [ ]:
!wget https://raw.githubusercontent.com/Rstam59/TaskDataRepoForStudents/refs/heads/main/housing.csv

In [ ]:
df1=pd.read_csv('housing.csv')
df1

In [ ]:
df1.isna().sum()

In [ ]:
df1.dtypes

In [ ]:
df1.info()

In [ ]:
df1.describe()

In [ ]:
df1.plot(kind='scatter',x='longitude',y='latitude',alpha=0.5,c='median_house_value',
         cmap='jet',
         figsize=(8,5),
         grid=True,
         s=df1['total_bedrooms']/50); # s means size

In [ ]:
df1.hist(figsize=(12,7));

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.subplot(2,1,1)
plt.hist(df1['total_bedrooms'],bins=50);
plt.subplot(2,1,2)
np.log(df1['total_bedrooms']).hist(bins=50);



In [ ]:
df1['total_bedrooms'].hist(bins=50);

In [ ]:
df1.corr(numeric_only=True)['median_house_value'].sort_values(ascending=False)

In [ ]:
X=df1.drop('median_house_value',axis=1)
y=df1['median_house_value'].copy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=(0.2),random_state=42)

In [ ]:
plt.hist(df1['median_income'],bins=50);

In [ ]:
df1['income_cat']=pd.cut(df1['median_income'],bins=[0.,1.5,3.0,4.5,6.0,np.inf], labels=[1,2,3,4,5])
df1['income_cat'].value_counts().sort_index().plot(kind='bar');

In [ ]:
X_train,X_test, y_train,y_test=train_test_split(X,y, stratify=df1['income_cat'] , test_size=0.2, random_state=42);

In [ ]:
df1.drop('income_cat',axis=1,inplace=True)

In [ ]:
df1['rooms_per_house'] = df1['total_rooms']/ df1['households']
df1['bedroom_ration'] = df1['total_bedrooms']/ df1['total_rooms']
df1['population_per_house'] = df1['population']/ df1['households']

In [ ]:
df1.corr(numeric_only=True)['median_house_value'].sort_values(ascending=False)

In [ ]:
df1.isna().sum()

In [ ]:
df1.drop('total_bedrooms',axis=1) # obtion1

In [ ]:
df1.dropna() # obtion 2

In [ ]:
X_num=X_train.select_dtypes(include=np.number)
X_cat=X_train.select_dtypes(include='object')
X_num


In [ ]:
X_cat

In [ ]:
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(strategy='median')
imputer.fit(X_num)

In [ ]:
imputer.statistics_

In [ ]:
X_num.median().values

In [ ]:
imputer.feature_names_in_

In [ ]:
pd.DataFrame(imputer.transform(X_num),columns=imputer.feature_names_in_)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder=OrdinalEncoder()
encoder.fit_transform(X_cat)

In [ ]:
encoder.categories_

In [ ]:
from sklearn.preprocessing import OneHotEncoder # unique deyerler coxsusaa onehotcoder isledilmir!

encoder=OneHotEncoder()
encoder.fit_transform(X_cat).toarray()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler= MinMaxScaler(feature_range=(-1,1))
pd.DataFrame(scaler.fit_transform(X_num) , columns=X_num.columns)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
pd.DataFrame(scaler.fit_transform(X_num), columns=scaler.feature_names_in_)

In [ ]:
from sklearn.preprocessing import FunctionTransformer

log_transformer = FunctionTransformer(np.log, inverse_func=np.exp)
log_pop = log_transformer.transform(X_num[['population']])

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline

X_num_pipeline=Pipeline([
     ('impute', SimpleImputer(strategy='median')),
     ('scale' ,StandardScaler()) ])


X_num=pd.DataFrame(X_num_pipeline.fit_transform(X_num),columns=X_num.columns)
X_num

In [ ]:
X_cat_pipeline = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OrdinalEncoder() )

X_cat = pd.DataFrame(X_cat_pipeline.fit_transform(X_cat), columns=X_cat_pipeline.feature_names_in_)

X_cat




In [ ]:
X_cat_pipeline= make_pipeline(
    SimpleImputer(strategy='most_frequent' ),  #fill_value='abc'
    OneHotEncoder() )


X_cat =pd.DataFrame(X_cat_pipeline.fit_transform(X_cat).toarray(),
                    columns= X_cat_pipeline.get_feature_names_out())

X_cat

In [ ]:
pd.concat([X_num,X_cat],axis=1) # to merge

In [ ]:
cat_columns = X_train.select_dtypes(include='object').columns
num_columns = X_train.select_dtypes(include= np.number).columns
num_columns

In [ ]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer([
    ('num', X_num_pipeline, num_columns),
    ('cat',X_cat_pipeline, cat_columns)
])

pd.DataFrame(transformer.fit_transform(X_train),columns=transformer.get_feature_names_out())

In [ ]:
# from sklearn.compose import ColumnTransformer

# num_cols=X_train.select_d

In [ ]:
X_train_prepared=pd.DataFrame(transformer.fit_transform(X_train),columns=transformer.get_feature_names_out())
X_test_prepared=pd.DataFrame(transformer.transform(X_test),columns=transformer.get_feature_names_out())

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg =LinearRegression()
lin_reg.fit(X_train_prepared,y_train)
lin_reg.score(X_test_prepared, y_test)

In [ ]:
lin_reg.predict(X_test_prepared.iloc[:5])

In [ ]:
y_test.iloc[:5]

In [ ]:
from sklearn.linear_model import LinearRegression

full_pipeline= make_pipeline(transformer,LinearRegression())
full_pipeline.fit(X_train,y_train)


In [ ]:
full_pipeline.score(X_test,y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

full_pipeline=make_pipeline(transformer,RandomForestRegressor())
full_pipeline.fit(X_train,y_train)
full_pipeline.score(X_test,y_test)

In [ ]:
full_pipeline.predict(X_test.iloc[:5])

In [ ]:
y_test.iloc[:5]

In [ ]:
from sklearn.metrics import mean_absolute_error

preds=full_pipeline.predict(X_test)
mean_absolute_error(y_test,preds)

In [ ]:
df1['median_house_value'].mean()

In [ ]:
def column_ratio(X):
  return X[:, [0]]/ X[:, [1]]

def ratio_name(function_transformer, feature_names_in):
  return["ration"]

def ratio_pipeline():
  return make_pipeline(
      SimpleImputer(strategy='median'),
      FunctionTransformer(column_ratio,feature_names_out=ratio_name),
      StandardScaler()

  )

log_pipeline=make_pipeline(
    SimpleImputer(strategy='median'),
    FunctionTransformer(np.log, feature_names_out='one-to-one'),
    StandardScaler()


)

cat_pipeline = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore')
)

default_num_pipeline= make_pipeline(SimpleImputer(strategy='median'),StandardScaler())

preprocessing= ColumnTransformer([
    ('bedrooms',ratio_pipeline(),['total_bedrooms','total_rooms']),
    ('rooms_per_house', ratio_pipeline(),['total_rooms','households']),
    ('people_per_house',ratio_pipeline(),['population','households']),
    ('log',log_pipeline,['total_bedrooms','total_rooms','population','households','median_income']),
    ('cat',cat_pipeline,['ocean_proximity'])
],
    remainder= default_num_pipeline)


In [ ]:
X_train_prepared=pd.DataFrame(preprocessing.fit_transform(X_train),columns=preprocessing.get_feature_names_out())
X_train_prepared

In [ ]:
full_pipeline=make_pipeline(preprocessing,LinearRegression())
full_pipeline.fit(X_train,y_train)
full_pipeline.score(X_test,y_test)

In [ ]:
full_pipeline=make_pipeline(preprocessing,DecisionTreeRegressor())
full_pipeline.fit(X_train,y_train)
full_pipeline.score(X_test,y_test)

In [ ]:
full_pipeline=make_pipeline(preprocessing,RandomForestRegressor())
full_pipeline.fit(X_train,y_train)
full_pipeline.score(X_test,y_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

full_pipeline=make_pipeline(transformer,DecisionTreeRegressor())
full_pipeline.fit(X_train,y_train)
full_pipeline.score(X_test,y_test)

In [ ]:
for i in range(100,170,10):
    full_pipeline=make_pipeline(preprocessing, RandomForestRegressor(n_estimators=i))
    full_pipeline.fit(X_train[:8000],y_train[:8000])
    print(i,full_pipeline.score(X_test,y_test))

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(full_pipeline, X_train[:8000], y_train[:8000],cv=5, scoring='neg_mean_absolute_error')

In [ ]:
X_train_prepared=pd.DataFrame(preprocessing.fit_transform(X_train),columns=preprocessing.get_feature_names_out())


In [ ]:
X_test_prepared=pd.DataFrame(preprocessing.transform(X_test),columns=preprocessing.get_feature_names_out())


In [ ]:
from sklearn.model_selection import GridSearchCV
params={
    'n_estimators':[100,120,140,160],
    'max_feature':[3,5,7,8]
}

grid_search = GridSearchCV(RandomForestRegressor(n_jobs=-1),params,cv=3,scoring='neg_mean_absolute_error')
grid_search.fit(X_train_prepared[:8000],y_train[:8000])

In [ ]:
grid_search.best_params_ # find the best combination

In [ ]:
grid_search.best_estimators_ # find the best estimator

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
params={
    'n_estimators': np.arange(100,250,10),
    'max_feature':[3,5,7,8]
}
random_search= RandomizedSearchCV(RandomForestRegressor(n_jobs=-1),
                                   params,cv=3,scoring='neg_mean_absolute_error',
                                  n_iter=15)

random_search.fit(X_train_prepared[:8000],y_train[:8000])


In [ ]:
random_forest_reg=random_search.best_estimator_
random_forest_reg.fit(X_train_prepared,y_train)
random_forest_reg.score(X_test_prepared,y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params={
    'randomforestregressor_n_estimators':[100,120,140,160],
    'randomforestregressor_max_feature': [3,5,7,8]  }

grid_search = GridSearchCV(full_pipeline,params,cv=3,scoring='neg_mean_absolute_error')
grid_search.fit(X_train,y_train)

In [ ]:
params={
    'randomforestregressor__n_estimators':[100,120,140,160], # 2 defe _ _ underscore qeyd olunur
    'randomforestregressor__max_feature': [3,5,7,8]  }

grid_search = GridSearchCV(full_pipeline,params,cv=3,scoring='neg_mean_absolute_error')
grid_search.fit(X_train,y_train)

In [ ]:
sorted(zip(random_forest_reg.feature_importances_, X_train_prepared.columns),reverse=True)

In [ ]:
from  sklearn.ensemble import IsolationForest

In [ ]:
iso_forest=IsolationForest(n_estimators=100,random_state=13)
iso_forest.fit(X_train_prepared)
iso_forest.predict(X_train_prepared)

In [ ]:
!kaggle datasets download -d "START-UMD/gtd"

In [ ]:
!unzip /content/gtd.zip

In [ ]:
task=pd.read_csv('globalterrorismdb_0718dist.csv',encoding='latin-1')
task

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
task1=task[['attacktype1_txt','nkill','country_txt','natlty1_txt','attacktype1']]
task1

In [ ]:
a=task1.groupby('country_txt').sum('nkill').reset_index().rename(columns={'nkill':'fatalities'})
b=a[['country_txt','fatalities']].sort_values(by='fatalities',ascending=False).head(10)

fig,ax=plt.subplots(figsize=(7,5))
bar=ax.bar(x=b['country_txt'], height=b['fatalities'], label='Fatalities');
ax.set(title='fatalities over countries',xlabel='country',ylabel='death number')
ax.legend(title='Data');
plt.xticks(rotation=45);  # Rotate x-axis labels for better visibility
plt.tight_layout();


In [ ]:
c=task1.groupby('natlty1_txt').sum('attacktype1').reset_index()
c=c[['natlty1_txt','attacktype1']].rename(columns={'natlty1_txt':'nationality','attacktype1':'attack_count'}).sort_values(by='attack_count',ascending=False).head(50)
fig,ax=plt.subplots(figsize=(10,7))
bar=ax.bar(x=c['nationality'],height=c['attack_count'],label='attack_count');
ax.set(title='attack count over nationality',xlabel='nation',ylabel='attack count');
ax.legend(title='data');
plt.xticks(rotation=45);
plt.tight_layout();

In [ ]:
d=task[['natlty1_txt','iyear','attacktype1','nkill']]
d

In [ ]:
d1=d.groupby('iyear').nunique('natlty1_txt').reset_index()
d2=d1[['iyear','natlty1_txt']].rename(columns={'natlty1_txt':'nationality_count_get_attackted'})
fig,ax=plt.subplots(figsize=(7,5))
plot=ax.plot(d2['iyear'],d2['nationality_count_get_attackted'],  label='nationality_count_get_attackted');
ax.set(title='attack number over year',xlabel='year',ylabel='attack count');
ax.legend(title='data');
plt.xticks(rotation=45);
plt.tight_layout();

In [ ]:
task2=task[['approxdate','imonth','iday','iyear','attacktype1','natlty1_txt']]
task2

In [ ]:
spring=task2[(task2['imonth']>=3) & (task2['imonth']<=5)]
summer=task2[(task2['imonth']>=6) & (task2['imonth']<=8)]
autumn=task2[(task2['imonth']>=9) & (task2['imonth']<=11)]
winter=task2[(task2['imonth']<=2) | (task2['imonth']==12)]

In [ ]:
s=spring.groupby('iyear')['attacktype1'].sum().reset_index()


In [ ]:
sum=summer.groupby('iyear')['attacktype1'].sum().reset_index()


In [ ]:
fall=autumn.groupby('iyear')['attacktype1'].sum().reset_index()


In [ ]:
win=winter.groupby('iyear')['attacktype1'].sum().reset_index()


In [ ]:
merged=s.merge(sum,on='iyear').merge(fall,on='iyear')
over_season=merged.rename(columns={'attacktype1_x':'spring','attacktype1_y':'summer','attacktype1':'autumn'}).merge(win,on='iyear').rename(columns={'attacktype1':'winter'})
over_season

In [ ]:
fig,ax=plt.subplots(figsize=(10,7))
plot=ax.plot(over_season['iyear'],over_season['spring'],label='spring',c='r');
plot=ax.plot(over_season['iyear'],over_season['summer'],label='summer',c='g');
plot=ax.plot(over_season['iyear'],over_season['autumn'],label='autumn',c='b');
plot=ax.plot(over_season['iyear'],over_season['winter'],label='winter',c='black');
ax.set(title='attack over seasons',xlabel='years',ylabel='attack counts');
ax.legend(title='info');

In [ ]:
task3=task[['region','region_txt','attacktype1','attacktype1_txt','targtype1','targtype1_txt','target1','iyear']]
task3

In [ ]:
big=task3.groupby('attacktype1')['iyear'].count().reset_index().rename(columns={'iyear':'count_attack'})
plt.bar(x=big['attacktype1'],height=big['count_attack'],label='number over attack type');
plt.legend();
# plt.title('Number over Attack Type');
# plt.xlabel('attack type');
# plt.ylabel('total count');

In [ ]:
must=task3.groupby(['targtype1_txt','iyear'])['region'].nunique('attacktype1').reset_index()
must

In [ ]:
val= task3.groupby(['iyear', 'targtype1_txt'])['attacktype1'].value_counts().to_frame('count').reset_index()
val1=val[['iyear','targtype1_txt','count']]
val1

In [ ]:
max_value=val1.groupby(['iyear', 'targtype1_txt'])['count'].sum().reset_index()
m=max_value.loc[max_value.groupby('iyear')['count'].idxmax()]
plt.plot(m['iyear'],m['count'],label='target level');
plt.title('target over year');
plt.xlabel('year');
plt.ylabel('count')
plt.legend();
plt.xticks(rotation=40);
plt.tight_layout();
plt.marker='s'


In [ ]:
fig,ax=plt.subplots(figsize=(7,5))
plot=ax.scatter(val1['iyear'],val1['targtype1_txt'])

In [ ]:
grouped_data = val1.groupby(['iyear', 'targtype1_txt'])['count'].sum().reset_index()
max_target_per_year = grouped_data.loc[grouped_data.groupby('iyear')['count'].idxmax()]
fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(max_target_per_year['iyear'], max_target_per_year['count'], marker='o', color='b')
ax.set_title('Most Frequent Target Type Over the Years')
ax.set_xlabel('Year')
ax.set_ylabel('Total Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
grouped_data = val1.groupby(['iyear', 'targtype1_txt'])['count'].sum().reset_index()
max_target_per_year = grouped_data.loc[grouped_data.groupby('iyear')['count'].idxmax()]
fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(max_target_per_year['iyear'], max_target_per_year['targtype1_txt'], marker='P', color='g')
ax.set_title('Most Frequent Target Type Over the Years')
ax.set_xlabel('Year')
ax.set_ylabel('Total Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
max_target_per_year = grouped_data.loc[grouped_data.groupby('iyear')['count'].idxmax()]
max_target_per_year

In [ ]:
grouped_data = val1.groupby(['iyear', 'targtype1_txt'])['count'].sum().reset_index()
grouped_data

In [ ]:
task4=task[['country','country_txt','region','region_txt','attacktype1_txt','iyear']]
task4

In [ ]:
value_region=task4.groupby(['iyear','region_txt']).size().to_frame('count').reset_index()
vas=value_region.groupby(['iyear','region_txt'])['count'].sum().reset_index()
vas

In [ ]:
vas=vas.loc[vas.groupby('iyear')['count'].idxmax()]
fig,ax=plt.subplots(figsize=(7,5))
plot=ax.plot(vas['iyear'],vas['region_txt'],label='terror level',marker='o', c='g');
ax.set(title='terror over region', xlabel='iyear',ylabel='count');
ax.legend();
plt.xticks(rotation=45);
plt.tight_layout();

In [ ]:
value_region1=task4.groupby(['iyear','region_txt']).size().to_frame('count').reset_index()
vas1=value_region.groupby('region_txt')['count'].sum().reset_index()
# vas1
plt.bar(x=vas1['region_txt'], height=vas1['count'],label='region terror')
plt.legend()
plt.xticks(rotation=45);
plt.tight_layout()


In [ ]:
task5=task[['attacktype1','attacktype1_txt','city','targtype1_txt','target1','nkill']]
task5

In [ ]:
value5=task5.groupby('targtype1_txt')['nkill'].mean().reset_index().sort_values('nkill',ascending=False)
val5=value5.head(10)
val5

In [ ]:
fig,ax=plt.subplots(figsize=(10,7))
pie=ax.pie(val5['nkill'],labels=val5['targtype1_txt'], autopct='%.2f%%');
ax.set(title='target type')

In [ ]:
task6=task[['iyear','attacktype1','attacktype1_txt']]
attack_type=task6.groupby('attacktype1_txt')['attacktype1'].count().to_frame('count').reset_index().sort_values(by='count',ascending=False)
attack_type

In [ ]:
plt.plot(attack_type['attacktype1_txt'],attack_type['count'],label='attack type indicator');
plt.title('attack type level');
plt.xlabel('attack type');
plt.ylabel('count');
plt.xticks(rotation=45);
plt.tight_layout();
plt.legend();

In [ ]:
task7=task[['iyear','guncertain1','weaptype1_txt','weapsubtype1_txt','weapdetail']]
task7

In [ ]:
weapon_value=task7.groupby('weaptype1_txt')['weaptype1_txt'].size().to_frame('count').reset_index().sort_values(by='count',ascending=False)
weapon_value

In [ ]:
fig,ax=plt.subplots(figsize=(10,7))
bar=ax.bar(height=weapon_value['count'],x=weapon_value['weaptype1_txt'],label='weapon type')
plt.xticks(rotation=45);
ax.legend();
plt.tight_layout;
ax.set(title='most used weapon type',xlabel='weapon type',ylabel='total count');

In [ ]:
xl=task7.groupby(['iyear','weaptype1_txt'])['weaptype1_txt'].count().to_frame('count').reset_index()
xl

In [ ]:
result=xl.loc[xl.groupby('iyear')['count'].idxmax()]
result

In [ ]:
fig,ax=plt.subplots(figsize=(8,5))
plot=ax.plot(result['iyear'],result['count'],label='weapon typyes', marker='s', color='r');
ax.set(title='weapon type over years',xlabel='year',ylabel='count');
ax.legend();
plt.xticks(rotation=45);
plt.tight_layout();

In [ ]:
task[['property','propextent','propextent_txt','propvalue']]

In [ ]:
task

In [ ]:
geo_viz = task[task['longitude'] >= -180]
geo_viz

In [ ]:
plt.subplots(figsize=(12, 7))
plt.scatter(x=geo_viz['longitude'], y=geo_viz['latitude'])